# **FireWorks/Atomate Tutorial (VASP)**

In [1]:
# Importing standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# ASE
import ase
from ase.build import molecule

# PMG
import pymatgen
from pymatgen.io.cif import CifParser
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import Structure
from pymatgen.core.surface import Slab
from pymatgen.io.vasp.sets import MVLSlabSet
from pymatgen.io.vasp.inputs import Kpoints
from pymatgen.analysis.elasticity.strain import Deformation
from pymatgen.core.surface import (
    SlabGenerator,
    get_symmetrically_distinct_miller_indices,
)
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.adsorption import AdsorbateSiteFinder

# MP Rester
from pymatgen.ext.matproj import MPRester
mpr = MPRester()

# FireWorks and Atomate
from pydash.objects import has, get
from atomate.vasp.fireworks.core import OptimizeFW, TransmuterFW, StaticFW
from atomate.vasp.config import VASP_CMD, DB_FILE
from fireworks import FiretaskBase, Firework, Workflow, explicit_serialize, LaunchPad
from fireworks.utilities.fw_serializers import DATETIME_HANDLER
from atomate.utils.utils import env_chk, get_logger
from atomate.vasp.database import VaspCalcDb
from fireworks.core.rocket_launcher import rapidfire

# Analysis and Theoretical level
from src.analysis import PlotEncutCalib, FitEOSTask, SlabThicknessTask, AdsorptionEnergyTask
from src.dft_settings import TheoreticalLevelSet, SelectiveDynamics

# Logger
logger = get_logger(__name__)

# Initialize LaunchPad and Reset
launchpad = LaunchPad()
launchpad.reset("", require_password=False)

2022-05-17 15:13:46,644 INFO Performing db tune-up
2022-05-17 15:13:46,677 INFO LaunchPad was RESET.


## **1.0 Theoretical level Calibration on Bulk Structure**

<center><img src="images/encut_calibration_wf.png" width=800 height=800 /></center>

In [ ]:
# Get the Pt bulk structure from Materials Project
bulk_structure = mpr.get_structure_by_material_id("mp-126", final=False, conventional_unit_cell=False)
print(bulk_structure)

In [ ]:
# Alternative from CIF file
bulk_structure = CifParser("Pt_mp-126.cif").get_structures(primitive=False)[0]
print(bulk_structure)

In [ ]:
# Cut-off Calibration
def ENCUT_WF(structure, encuts, vasp_cmd=VASP_CMD, db_file=DB_FILE):
    
    # List of fireworks
    fws, parents = [], []
    
    # Loop over each ENCUT value
    for n, encut in enumerate(encuts):
        
        # Create dict with new ENCUT key
        incar_settings = {"ENCUT": float(encut), "NSW": 0}
        
        # Apply settings
        vis_static_encut = TheoreticalLevelSet(structure, 
                                               bulk=True,
                                               user_incar_settings=incar_settings)
        
        # Create a Static (single point) calculation
        if n == 0:
            fw = OptimizeFW(name=f"Static-{encut}-{n}", 
                          structure=structure,
                          max_force_threshold=None,  
                          vasp_input_set=vis_static_encut,
                          job_type="normal",
                          vasp_cmd=vasp_cmd,
                          db_file=db_file)
            fws.append(fw)
            
        else:
            fw = OptimizeFW(name=f"Static-{encut}-{n}", 
                          structure=structure,
                          max_force_threshold=None,
                          vasp_input_set=vis_static_encut,
                          job_type="normal",
                          vasp_cmd=vasp_cmd,
                          db_file=db_file, parents=fws[n-1])
            fws.append(fw)
            
    # Post-processing
    parents = fws[-1]
    fw_plot = Firework(PlotEncutCalib(db_file=db_file), name="Encut calibration Plot", parents=parents)
    fws.append(fw_plot)
    
    # Wrap into a Workflow
    wf_encut = Workflow(fws, name="Pt ENCUT Calibration")
    return wf_encut


In [ ]:
# Launch and Run!
encuts_list = [280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500]

wf_encut = ENCUT_WF(structure=bulk_structure, encuts=encuts_list)
launchpad.add_wf(wf_encut)

rapidfire(launchpad)

## **2.0 Bulk Structure (Equation of States)**

<center><img src="images/eos_wf.png" width=1500 height=1500 /></center>

In [ ]:
# Create EOS_WF
def EOS_WF(structure, deformations, eos="vinet", vasp_cmd=VASP_CMD, db_file=DB_FILE):
    
    # INCAR settings (relax both positions and cell)
    relax_incar_settings = {"ENCUT": 420, "ISIF": 3}
    
    # Bulk optimization
    vis_relax = TheoreticalLevelSet(structure, 
                                    bulk=True,
                                    user_incar_settings=relax_incar_settings)
    
    # Structure optimization firework
    fws = [OptimizeFW(structure=structure,
                      vasp_input_set=vis_relax, 
                      max_force_threshold=None,
                      job_type="normal",
                      vasp_cmd=vasp_cmd, 
                      db_file=db_file, name="structure_optimization")]
    
    # Static settings for deformations
    static_incar_settings = {"ENCUT": 420, "NSW": 0}
    vis_static = TheoreticalLevelSet(structure,
                                     bulk=True,
                                     user_incar_settings=static_incar_settings)
    
    # Create each deformation Firework and add them to the FW list
    parents = fws[0]
    deformations = [Deformation(defo_mat) for defo_mat in deformations]
    for n, deformation in enumerate(deformations):
        fw = TransmuterFW(name=f"bulk_deformation_{n}",
                          structure=structure,
                          transformations=["DeformStructureTransformation"],
                          transformation_params=[{"deformation": deformation.tolist()}],
                          vasp_input_set=vis_static, parents=parents,
                          vasp_cmd=vasp_cmd, db_file=db_file)
        fws.append(fw)
        
    # Post-processing
    parents = fws[1:]
    fw_analysis = Firework(FitEOSTask(eos=eos, db_file=db_file, to_db=True), name="EOS Fitting Analysis", parents=parents)
    fws.append(fw_analysis)
    
    # Wrap into a workflow
    wf_eos = Workflow(fws, name="EOS Workflow")
    return wf_eos    

In [ ]:
# Launch and Run!
deformations = [(np.identity(3) * (1 + x)).tolist() for x in np.linspace(-0.12, 0.12, 11)]

wf_eos = EOS_WF(structure=bulk_structure, deformations=deformations)
launchpad.add_wf(wf_eos)

rapidfire(launchpad)

## **3.0 From Pt Bulk -> Pt (111) Surface**

<center><img src="images/slab_thickness_wf.png" width=1500 height=1500 /></center>

In [ ]:
# Retrieve Equilibrium bulk structure from the DB
def get_eq_bulk_db():
    
    # Path to DB_FILE
    db_file = '/home/jovyan/atomate/config/db.json'
    
    # Connect to the DB
    mmdb = VaspCalcDb.from_db_file(db_file, admin=True)
    
    # Search the EOS collection created before
    collection = mmdb.db["eos"]
    
    # Access to docs
    doc = collection.find({})[0]
    
    # Equilibrium structure
    struct = Structure.from_dict(doc["structure_eq"])
    
    return struct

bulk_struct_eq = get_eq_bulk_db()
bulk_struct_eq.to(filename="Pt_mp-126.cif")
print(bulk_struct_eq)

In [ ]:
# Alternative from CIF file
bulk_struct_eq = CifParser("Pt_mp-126_eq.cif").get_structures(primitive=False)[0]
print(bulk_struct_eq)

In [ ]:
# Slab Thickness Calibration
def SlabThickness_WF(structure, miller_index, layers, repeat=[2,2,1], vasp_cmd=VASP_CMD, db_file=DB_FILE):
    
    # Given a PMG structure object get conventional standard
    SGA = SpacegroupAnalyzer(structure)
    bulk_structure = SGA.get_conventional_standard_structure()
    
    # Get the slab model
    slab_list = []
    for n, layer in enumerate(layers):
        # Generate the slab model
        slab_gen = SlabGenerator(
                        bulk_structure,
                        miller_index=miller_index,
                        min_slab_size=layer,
                        min_vacuum_size=12,
                        in_unit_planes=True,
                        center_slab=True,
                        reorient_lattice=True,
                        lll_reduce=True)
        # Symmetrize
        slab = slab_gen.get_slabs(symmetrize=True, ftol=0.01)[0]
        # make supercell
        slab.make_supercell(repeat)
        # selective dynamics
        slab_new = SelectiveDynamics.center_of_mass(slab)
        # Append to slab_list
        slab_list.append(slab_new)
        
    # Theoretical method for oriented bulk
    vis_bulk = TheoreticalLevelSet(structure=slab_list[0].oriented_unit_cell,
                                   bulk=True, user_incar_settings={"ENCUT": 420, "NSW": 0})
    
    fws = [OptimizeFW(name=f"oriented_bulk",
                         structure=slab_list[0].oriented_unit_cell,
                         vasp_input_set=vis_bulk,
                         max_force_threshold=None,
                         job_type="normal",
                         vasp_cmd=vasp_cmd,
                         db_file=db_file)]
    
    # OptimizeFW for each surface
    for layer, slab in zip(layers, slab_list):
        
        # Theoretical level for each thickness
        vis_slab_clean = TheoreticalLevelSet(structure=slab,
                                             bulk=False,
                                             user_incar_settings={"ENCUT": 420, "NELM": 80})
        # OptimizeFW 
        fw = OptimizeFW(name=f"slab_thickness_{layer}",
                        structure=slab,
                        vasp_input_set=vis_slab_clean,
                        max_force_threshold=None,
                        job_type="normal",
                        vasp_cmd=vasp_cmd,
                        db_file=db_file, parents=fws[0])
        # Append
        fws.append(fw)
            
    # Post-processing slab_thickness
    parents = fws[1:]
    fw_analysis = Firework(SlabThicknessTask(db_file=db_file, to_db=True), name="Slab Thickness Analysis", parents=parents)
    fws.append(fw_analysis)
    
    # Wrap into a workflow
    wf_thick = Workflow(fws, name="Slab Thickness Workflow")
    return wf_thick

In [ ]:
# Launch and Run!
layers_list = [2, 3, 4, 5, 6]

wf_thick = SlabThickness_WF(structure=bulk_struct_eq, miller_index=(1,1,1), layers=layers_list[::-1])
launchpad.add_wf(wf_thick)

rapidfire(launchpad)

In [2]:
# Helper function to retrieve thickness collection and select best slab thickness
def get_slab_thickness(thickness=4):
    from pymatgen.core.surface import Slab
    
    # Path to DB_FILE
    db_file = '/home/jovyan/atomate/config/db.json'
    
    # Connect to the DB
    mmdb = VaspCalcDb.from_db_file(db_file, admin=True)
    
    # Search the EOS collection created before
    collection = mmdb.db["thickness"]
    
    # Access to docs
    doc = collection.find({})[0]
    
    # slab_obj_dict
    slab_objects = doc["slab_objs"]
    
    # slab
    slab = Slab.from_dict(slab_objects[str(thickness)])
    
    return slab

# Get the slab object from thickness collection
pt_111_slab = get_slab_thickness()

## **4.0 Surface+Adsorbate (Adsorption Energy)**

<center><img src="images/adsorption_energy_wf.png" width=1500 height=1500 /></center>

In [3]:
# Adsorption energy Workflow
def AdsorptionE_WF(slab, thickness=4, adsorbate="CO", ads_cell=[15, 15, 15], positions=["ontop", "bridge", "hollow"], vasp_cmd=VASP_CMD, db_file=DB_FILE):
    
    # PMG ASEAtomsAdaptor
    ase_adpt = AseAtomsAdaptor()
    
    # Adsorbate in a Box
    ads_box = molecule(adsorbate)
    ads_box.set_cell(ads_cell)
    ads_box.center()
    ads_struct = ase_adpt.get_structure(ads_box)
    
    # Theoretical level for the adsorbate in a box 
    vis_ads_box = TheoreticalLevelSet(ads_struct, 
                                      bulk=True, 
                                      user_incar_settings={"ENCUT": 420},
                                      user_kpoints_settings=Kpoints.gamma_automatic((1,1,1)))
    
    # OptimizeFW for the ads in a box
    fws = [OptimizeFW(name=f"adsorbate_box", 
                      structure=ads_struct, 
                      vasp_input_set=vis_ads_box,
                      max_force_threshold=None,
                      job_type="normal",
                      vasp_cmd=vasp_cmd,
                      db_file=db_file)]
    
    # Adsorbate as Molecule object
    ads_mol = molecule(adsorbate)
    ads_mol = ase_adpt.get_molecule(ads_mol)
    
    # AdsorbateSiteFinder
    asf = AdsorbateSiteFinder(slab)
    slab_sites = asf.find_adsorption_sites(positions=positions, put_inside=True)
    
    # Loop over sites
    slab_ads_dict = {}
    for key, value in slab_sites.items():
        if key == "ontop":
            slab_ads = asf.add_adsorbate(ads_mol, value[0])
            slab_ads_sd = SelectiveDynamics.center_of_mass(slab_ads)
            slab_ads_dict.update({str(key): slab_ads_sd.as_dict()})
            
        if key == "bridge":
            slab_ads = asf.add_adsorbate(ads_mol, value[0])
            slab_ads_sd = SelectiveDynamics.center_of_mass(slab_ads)
            slab_ads_dict.update({str(key): slab_ads_sd.as_dict()})
            
        if key == "hollow":
            slab_ads = asf.add_adsorbate(ads_mol, value[0])
            slab_ads_sd = SelectiveDynamics.center_of_mass(slab_ads)
            slab_ads_dict.update({str(key): slab_ads_sd.as_dict()})
            
    # OptimizeFW for each slab_ads
    for n, (label, slab_ads) in enumerate(slab_ads_dict.items()):
        
        # PMG Slab
        slab_ads = Slab.from_dict(slab_ads)
        
        # Theoretical level
        vis_slab_ads = TheoreticalLevelSet(structure=slab_ads,
                                           bulk=False,
                                           user_incar_settings={"ENCUT": 420, "NSW": 800})
        # OptimizeFW
        fw = OptimizeFW(name=f"slab_ads_{label}_{n}",
                        structure=slab_ads,
                        vasp_input_set=vis_slab_ads,
                        max_force_threshold=None,
                        job_type="normal",
                        vasp_cmd=vasp_cmd,
                        db_file=db_file, parents=fws[0])
        # Append
        fws.append(fw)
        
    # Post-processing (Adsorption energy)
    parents = fws[1:]
    fw_analysis = Firework(AdsorptionEnergyTask(thickness=thickness, db_file=db_file, to_db=True), name="Adsorption Energy Analysis", parents=parents)
    fws.append(fw_analysis)
    
    # Wrap into a workflow
    wf_ads_e = Workflow(fws, name="Adsoption Energy Workflow")
    return wf_ads_e

In [4]:
# Launch and Run!
wf_ads_e = AdsorptionE_WF(slab=pt_111_slab, adsorbate="CO")
launchpad.add_wf(wf_ads_e)

rapidfire(launchpad)

2022-05-17 15:14:13,508 INFO Added a workflow. id_map: {-5: 1, -4: 2, -3: 3, -2: 4, -1: 5}
2022-05-17 15:14:13,525 INFO Created new dir /home/jovyan/launcher_2022-05-17-15-14-13-525060
2022-05-17 15:14:13,526 INFO Launching Rocket
2022-05-17 15:14:13,552 INFO RUNNING fw_id: 5 in directory: /home/jovyan/launcher_2022-05-17-15-14-13-525060
2022-05-17 15:14:13,561 INFO Task started: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}.
2022-05-17 15:14:13,795 INFO Task completed: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}
2022-05-17 15:14:13,799 INFO Task started: {{atomate.vasp.firetasks.run_calc.RunVaspCustodian}}.
2022-05-17 15:15:03,718 INFO Task completed: {{atomate.vasp.firetasks.run_calc.RunVaspCustodian}}
2022-05-17 15:15:03,727 INFO Task started: {{atomate.common.firetasks.glue_tasks.PassCalcLocs}}.
2022-05-17 15:15:03,729 INFO Task completed: {{atomate.common.firetasks.glue_tasks.PassCalcLocs}}
2022-05-17 15:15:03,732 INFO Task started: {{atomate.vasp.firet

ERROR:custodian.custodian:UnconvergedErrorHandler
/opt/vasp.6.2.0_pgi_mkl/bin/vasp_std: no process found


2022-05-17 16:56:00,105 INFO Rocket finished


Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/fireworks/core/rocket.py", line 261, in run
    m_action = t.run_task(my_spec)
  File "/opt/conda/lib/python3.8/site-packages/atomate/vasp/firetasks/run_calc.py", line 299, in run_task
    c.run()
  File "/opt/conda/lib/python3.8/site-packages/custodian/custodian.py", line 367, in run
    self._run_job(job_n, job)
  File "/opt/conda/lib/python3.8/site-packages/custodian/custodian.py", line 453, in _run_job
    time.sleep(self.polling_time_step)
KeyboardInterrupt
INFO:rocket.launcher:Rocket finished
DEBUG:launchpad:FW with id: 3 is unique!


2022-05-17 16:56:00,122 INFO Created new dir /home/jovyan/launcher_2022-05-17-16-56-00-121970


...

INFO:rocket.launcher:Task completed: {{atomate.vasp.firetasks.write_inputs.WriteVaspFromIOSet}}


2022-05-17 16:56:05,561 INFO Task started: {{atomate.vasp.firetasks.run_calc.RunVaspCustodian}}.


INFO:rocket.launcher:Task started: {{atomate.vasp.firetasks.run_calc.RunVaspCustodian}}.


2022-05-17 16:56:09,856 INFO Rocket finished


Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/fireworks/core/rocket.py", line 261, in run
    m_action = t.run_task(my_spec)
  File "/opt/conda/lib/python3.8/site-packages/atomate/vasp/firetasks/run_calc.py", line 299, in run_task
    c.run()
  File "/opt/conda/lib/python3.8/site-packages/custodian/custodian.py", line 367, in run
    self._run_job(job_n, job)
  File "/opt/conda/lib/python3.8/site-packages/custodian/custodian.py", line 453, in _run_job
    time.sleep(self.polling_time_step)
KeyboardInterrupt
INFO:rocket.launcher:Rocket finished
DEBUG:launchpad:Aggregation '[{'$match': {'$or': [{'spec._fworker': {'$exists': False}}, {'spec._fworker': None}, {'spec._fworker': 'atomate_stack'}], 'state': {'$in': ['RUNNING', 'RESERVED']}}}, {'$project': {'fw_id': True, '_id': False}}]'.


## **The End!**